In [1]:
from crossword_solver.crossword_detection import detect_crossword_from_file
from crossword_solver.solver_helpers import find_whole_crossword_candidates, reorder_crossword_hints, solving_algorithm
from crossword_solver.solver_helpers import find_suitable_candidates
from copy import deepcopy
from collections import defaultdict
import numpy as np


Resource 'estwordnet_2023-07-20' has already been downloaded.
INFO:keyedvectors.py:2047: loading projection weights from ../data/lemmas.cbow.s100.w2v.bin
INFO:utils.py:448: KeyedVectors lifecycle event {'msg': 'loaded (441391, 100) matrix of type float32 from ../data/lemmas.cbow.s100.w2v.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-05-07T23:49:58.596982', 'gensim': '4.3.2', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}


In [2]:
path = '../data/pictures/ristsona_2.png'
crossword = detect_crossword_from_file(path)


In [3]:
find_whole_crossword_candidates(crossword)

In [5]:
def solve_crossword(crossword, max_empty_words = 8, min_score = 10, min_intersections = 10):
    results = []
    solving_cw = deepcopy(crossword)
    new_hints = []
    for hint in solving_cw.hints:
        if len(hint.candidates)>0:
            new_hints.append(hint)
    solving_cw.hints = new_hints
    reorder_crossword_hints(solving_cw)
    for matrix, score, intersections in solving_algorithm(solving_cw, max_empty_words=max_empty_words):
        if score > min_score and intersections>min_intersections:
            results.append((matrix, score, intersections))
    return results




In [6]:
results = solve_crossword(crossword)


In [7]:
printing_crossword = deepcopy(crossword)
for matrix, score, intersections in sorted(results, key=lambda x: x[2], reverse = True)[:5]:
    printing_crossword.matrix = matrix
    print(printing_crossword, round(score, 1), intersections)

■ ■ ■ ■ ■ ■ 1 1 1 1 1
■ ■ ■ ■ ■ 1 o p a a l
■ ■ ■ ■ ■ 1 s r ü 2 _
■ ■ ■ ■ ■ 1 l i 2 v k
■ ■ ■ ■ ■ 1 _ s k a _
■ 1 1 1 1 1 2 m e s i
2 _ _ n t _ a a v a _
_ _ _ a a _ n 2 a r e
1 _ _ a i _ d _ d 2 e
1 f 2 b m _ 2 l e e r
1 l o e 2 _ _ _ 2 n i
1 e i r a m a 2 u _ _
1 j a 2 _ _ _ l u _ _
1 _ _ l _ _ _ n s ■ ■
 19.0 15
■ ■ ■ ■ ■ ■ 1 1 1 1 1
■ ■ ■ ■ ■ 1 o p a a l
■ ■ ■ ■ ■ 1 s r ü 2 _
■ ■ ■ ■ ■ 1 l i 2 v k
■ ■ ■ ■ ■ 1 _ s k a _
■ 1 1 1 1 1 2 m e s i
2 _ _ n t _ a a v a _
_ _ _ a a _ n 2 a r e
1 _ _ a i _ d _ d 2 e
1 c 2 b m _ 2 l e e r
1 l o e 2 _ _ _ 2 n i
1 e i r a m a 2 u _ _
1 j a 2 _ _ _ l u _ _
1 _ _ l _ _ _ n s ■ ■
 18.7 15
■ ■ ■ ■ ■ ■ 1 1 1 1 1
■ ■ ■ ■ ■ 1 o p a a l
■ ■ ■ ■ ■ 1 s r ü 2 _
■ ■ ■ ■ ■ 1 l i 2 v k
■ ■ ■ ■ ■ 1 _ s k a _
■ 1 1 1 1 1 2 m e s i
2 _ _ n t _ a a v a _
_ _ _ a a _ n 2 a r e
1 _ _ a i _ d _ d 2 e
1 b 2 b m _ 2 l e e r
1 l o e 2 _ _ _ 2 n i
1 e i r a m a 2 u _ _
1 j a 2 _ _ _ l u _ _
1 _ _ l _ _ _ n s ■ ■
 18.7 15
■ ■ ■ ■ ■ ■ 1 1 1 1 1
■ ■ ■ ■ ■ 1 o p a a l
■ ■ ■

In [ ]:
min_score = 5
results = list()
for matrix, score in solving_algorithm(crossword, max_empty_words=0):
    if score > min_score:
        results.append((matrix, score))

In [ ]:
a = 1
for hint in crossword.hints:
    a*=(len(hint.candidates)+1)
    print(hint.hint, hint.length, len(hint.candidates))
a

In [ ]:

def solving_algorithm(crossword, max_empty_words = 5):
    if len(crossword.hints)==0:
        yield np.copy(crossword.matrix), crossword.score, crossword.intersections
        return
    
    hint = crossword.hints.pop(0)
    suitable_candidates = find_suitable_candidates(hint, crossword)

    if len(suitable_candidates)==0 and max_empty_words==0:
        yield np.copy(crossword.matrix), crossword.score, crossword.intersections
    if max_empty_words>0:
        yield from solving_algorithm(crossword, max_empty_words-1)
    
    prev_text = np.copy(crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1])
    intersection_count = np.count_nonzero(prev_text != crossword.unfilled_character)
    crossword.intersections += intersection_count
    for candidate in suitable_candidates:
        crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1] = np.array(list(candidate.text)
                                                                                          ).reshape(prev_text.shape)
        crossword.score += candidate.weight
        yield from solving_algorithm(crossword, max_empty_words)
        crossword.score -= candidate.weight
        crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1] = prev_text
    crossword.intersections -= intersection_count
    crossword.hints.insert(0, hint)

In [ ]:

max_empty_words = 8
min_score = 1
min_intersections = 14
results = []
solving_cw = deepcopy(crossword)
solving_cw.intersections = 0
new_hints = []
for hint in solving_cw.hints:
    if len(hint.candidates)>0:
        new_hints.append(hint)
solving_cw.hints = new_hints
reorder_crossword_hints(solving_cw)
for matrix, score, intersections in solving_algorithm(solving_cw, max_empty_words=max_empty_words):
    if score > min_score and intersections>min_intersections:
        results.append((matrix, score, intersections))
        
print(len(results))
#print(len(solving_cw.hints))

In [ ]:
printing_crossword = deepcopy(crossword)
for matrix, score, intersections in sorted(results, key=lambda x: x[2], reverse = True)[:5]:
    printing_crossword.matrix = matrix
    print(printing_crossword, round(score, 1), intersections)

In [ ]:
asd = np.full((3, 1), "_")
asd[1, 0] = "A"
display(asd)
np.count_nonzero(asd != "_")
